# Retrieval Augmented Generation using LangChain
-Load enviroment variables into the main program

%pip install -qU pypdf

In [2]:
%pip install -qU pypdf


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

# Load enviroment file
load_dotenv()

True

Loading The Document

In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "data/book_abt_chairs.pdf"

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

print(pages[3].page_content)

8 
Методи на изследването 
При преследване на така поставената цел и в търсене на 
решение на произтичащите от нея задачи е необходи мо да 
подберем подходяща методология. За целта се ориентираме 
към използването на следните методи: 
 Анализ на литературните източници, свързани с 
разработваната проблематика – научни трудове, 
енциклопедии, периодични издания, каталози на 
фирми-производителки на мебели за седене и др.; 
 Системен подход, който стои в основата на 
сравнителен анализ по предварително избрани 
показатели на комплекс от фактори, разкрити при 
изследването или изложени по аналогични теми; 
 Семиотични анализи и мисловен експеримент; 
 Лични наблюдения и критичен анализ. 
Обхват на изследването 
Като се има предвид голямото разнообразие във 
функционалното предназначение на мебелите за седене, е 
необходимо, при дефиниране на обекта на изследване в 
настоящия труд, да се уточни, че в монографията се изследват 
тези столове, обслужващи функциите „хранене” и 
„индивидуал

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vector_store = InMemoryVectorStore.from_documents(pages, OpenAIEmbeddings())
docs = vector_store.similarity_search("Цветове", k=2)
for doc in docs:
    print(f'Page {doc.metadata["page"]}: {doc.page_content[:300]}\n')

Page 87: 92 
цветовете, композиционната изява на пропорциите, 
разчленяването на формата, ритъма, контраста.  
Тенденциите за новото хилядолетие диктуват на 
производителите да използват много цветове, пластични 
форми и качествени материали с цел постигане на по -добра 
психологическа атмосфера в съвременно

Page 86: 91 
5. Цвят, текстура и фактура 
Цветът, текстурата и фактурата като свойства на формата са 
неразделна част от визуалното възприемане на мебелите. Те 
могат да варират в широки граници. Текстурата на 
повърхността зависи от начина на обработката на 
материала. Цветът и текстурата не винаги характер

